<span>
<img align="right" width="320" src="http://www.sobigdata.eu/sites/default/files/SoBigData_RI_3.png">
</span>
<span>
<b>Author:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti and Katherine Abramski</a><br/>
<b>Python version:</b>  3.9<br/>
<b>Last update:</b> 19/09/2023
</span>

<a id='top'></a>
# *Mixing R and Python for Statistical Analyses*
This notebook contains an introduction to rpy2, a Python package that allows a basilar integration of R and Python, as well as some examples that demonstrate how to approach some statistical analyses.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started.

## Table of Contents
1. [Integrating R and Python](#R)
    1. [Push: Passing Variables from Python to R](#i)
    2. [Pull: Passing Variables from  R to Python](#o)
2. [Statistics Fundamentals](#rev)
    1. [The Normal Distribution](#normal)
    2. [Hypothesis Testing](#hyp)
3. [Statistical Tests](#tests)
    1. [Is there a relationship between part of speech (noun/verb) and gender?](#t_test)
    2. [Are polysemous words more concrete than non-polysemous words?](#mw_test)
    3. [Is there a relationship between part of speech (noun, verb, adjective) and polysemy?](#Chi)
    4. [Can imageability be used to predict word concreteness?](#reg)

In [ ]:
# load the python libraries

from functools import partial
from rpy2.ipython import html
import numpy as np
import pandas as pd
from scipy.stats import norm
import nltk
import matplotlib.pyplot as plt
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

In [ ]:
# Load the %%R magic 
%load_ext rpy2.ipython

In [ ]:
%%R

# Install R packages that we will use
# You only need to do this once

install.packages("tidyverse")
install.packages("stats")
install.packages("ggplot2")
install.packages("corrplot")
install.packages("MASS")

In [ ]:
%%R

# Load R packages that we will use
library(tidyverse)
library(stats)
library(ggplot2)
library(corrplot)
library(MASS)

<a id='R'></a>
## 1. Integrating R and Python ([to top](#top))

To execute an R script in jupyter notebook, it is mandatory to specify that the cell does not contain Python code.

Thic can be done by using "%%R" in the first line. Everything that follows should use the R syntax.

In [ ]:
%%R # This specifies that the following code is R code, not python code

x <- "This is R code!"

print(x)

In [ ]:
x = "This is python code!"

print(x)

Once a notebook is designed to contain both Python and R cells it is possible to move computation results across the two interpreters using variable push/pull strategies.

<a id='i'></a>
### 1.A Push: Passing Variables from Python to R ([to top](#top))

Let us assume we have some data in python, stored in a dataframe, and we want to move it to R.

In [ ]:
# Create a simple dataframe

python_df = pd.DataFrame({'A': [1, 2, 3],
                          'B': [4, 5, 6]},
                          )  
python_df

Mapping the dataframe to the R environment (where it will be stored as an R dataframe) is called a "push" and can be achieved using "-i" followed by the name of the variable to push in the %%R magic

In [ ]:
%%R -i python_df # pass the dataframe to R using "-i" followed by the variable name

python_df # this a a R dataframe now

<a id='o'></a>
### 1.B Pull: Passing Variables from  R to Python ([to top](#top))

Now let's assume we want to manipulate the dataframe in some way, and then send it back to R.

Moving an object from R to Python is called a "pull" and can be achieved using "-o" follwed by the name of the variable

In [ ]:
%%R -o python_df2
# python_df variable is still in R
# let's add a new column to the dataframe (the sum of A and B), then push it to back python

python_df2 <- python_df %>% mutate(C = A + B)
python_df2

Now the dataframe is back in python. Objects from the R environment are mapped either in a pandas dataframe (if possible), or in dictionaries (for complex objects like model outputs that cannot be mapped in a pandas dataframe)

In [ ]:
python_df2

<a id='rev'></a>
## 2. Statistics Fundamentals ([to top](#top))

Now let's review some basic statistical fundamentals: The Normal Distrubution, and Hypothesis Testing

<a id='normal'></a>
### 2.A The Normal Distribution ([to top](#top))

Let's start by looking at the distributions of men's and women's heights in Italy (using R).

We know men's and women's heights in Italy are approximately **normally distributed** such that:
- Men have a mean height of **175cm** with a standard deviation of **6cm**
- Women have a mean height of **165cm** with a standard deviation of **5cm**

These values reflect the population **parameters**.

In [ ]:
%%R

set.seed(100)
# Sample size
n <- 1000

# Height of Men
M_mean <- 175
M_SD <- 7
M <- rnorm(n, M_mean, M_SD)

# Height of Women
W_mean <- 165
W_SD <- 6
W <- rnorm(n, W_mean, W_SD)

print(round(head(M),1))
print(round(head(W),1))

Now from the samples let's:
- calculate the **sample means** and **sample standard deviations**
- view the **empirical** distributions of the sampled heights

What happens to the sample means and sample standard deviations as we increase 

In [ ]:
%%R

# Calculate means and SDs
print(paste0("M: sample mean = ", round(mean(M), 3), ", sample SD = ", round(sd(M), 3)))
print(paste0("W: sample mean = ", round(mean(W), 3), ", sample SD = ", round(sd(W), 3)))

# plot distributions
par(mfrow = c(1,2))
hist(M, breaks = 20, col = "purple")
hist(W, breaks = 20, col = "yellowgreen")

#### What information can we get from a data distribution?
A data distribution can give us important information about the probability of observing certain values.

**What percentage of the sample of Italian women is at least 170cm tall?**

We can get this number by simply counting the number of Italian women taller than 170cm in the sample and dividing by the total number in the sample.

In [ ]:
%%R
emp_p170 <- sum(W >= 170)/n * 100
emp_p170

From our **empirical distribution**, we find that the probability that an Italian woman is at least 170cm tall is **19.7%**.

But since we know the population **parameters** of Italian women's heights (mean = **165cm** and standard deviation = **6cm**) we can calculate the **probability that an Italian woman is at least 170cm** from the **theoretical distribution**.

This is equivalent to the area under the curve of the probability distribution function (pdf) of the standard normal distribution with mean = 165 and SD = 5 to the right of 176cm.

In [ ]:
%%R

teo_p170 <- pnorm(170, W_mean, W_SD, lower.tail = FALSE) * 100
print(teo_p170)
# lower.tail = FALSE because we want the area to the RIGHT of 170
# If we want the area to the LEFT of 170, we write lower.tail = TRUE

# View the probability distribution function (pdf)
x <- seq(from = W_mean - W_SD*4, to = W_mean + W_SD*4, by = 0.05)
norm_dat <- data.frame(dist = "N(1,4)", x = x, pdf = dnorm(x, mean = W_mean, sd = W_SD))
ggplot(norm_dat) + geom_line(aes(x = x, y = pdf)) + geom_vline(xintercept = 170, linetype = "dashed")

From our **theoretical distribution** we find that the probability that an Italian woman is at least 170cm is **20.2%**.

<a id='hyp'></a>
### 2.B Hypothesis Testing ([to top](#top))

Hypothesis testing is used to make inferences about a population using sampled data. Usually this involves testing some hypothesis about the relationship between variables of the population.

To get the intuition of how hypothesis testing works, let's try to make an inference about whether or not Katie is Italian based on her height.

#### Is Katie Italian?

**Observed data:** Katie is 176cm tall.

**Null Hypothesis:** $H_{0}$: Katie is Italian

**Alternative Hypothesis:** $H_{A}$: Katie in not Italian

**What is the probability that Katie is 176cm tall, assuming she is Italian?**

In other words...

What is the **probability that an Italian woman is at least 176cm?**

In [ ]:
%%R

teo_p176 <- pnorm(176, W_mean, W_SD, lower.tail = FALSE) * 100
print(teo_p176)
# lower.tail = FALSE because we want the area to the RIGHT of 170
# If we want the area to the LEFT of 170, we write lower.tail = TRUE

# View the probability distribution function (pdf)
x <- seq(from = W_mean - W_SD*4, to = W_mean + W_SD*4, by = 0.05)
norm_dat <- data.frame(dist = "N(1,4)", x = x, pdf = dnorm(x, mean = W_mean, sd = W_SD))
ggplot(norm_dat) + geom_line(aes(x = x, y = pdf)) + geom_vline(xintercept = 176, linetype = "dashed")

From our **theoretical distribution** reflecting the **null hypothesis** we find that the probability that an Italian woman is at least 176cm is **3.3%**.

This value is the **p-value: the probability of observing the data assuming the null hypothesis is true.**

Since the probability of observing the data (Katie is 176cm tall) assuming the null hypothesis is true (Katie is Italian) is very small (less than 5%) we say that we have sufficient evidence to **reject the null hypothesis.**

Therefore, we infer that Katie is not Italian.

In fact, she is American.

#### How can hypothesis testing be applied to make inferences about populations from sampled data?

The previous example was just to provide some intuition about what hypothesis testing is all about, specifically, how distributions can be used to calculate p-values to determine the significance of relationships. In the next part, we take a look at some examples of approaches to questions about relationships between variables using statistical tests.

<a id='tests'></a>
## 3. Statistical Tests ([to top](#top))

Statistical tests quantify the significance of **relationships between variables** of a population **inferred from sampled data**

**IMPORTANT:** The type of statistical test that should be used depends mainly on:
- the type of predictor and outcome variables (which help us determine our null and alternative hypotheses)
- certain assumptions about the sampled data (which help us determine whether to use a parametric or non-parametric test)

Let's have a look at choosing the right statistical test for finding relationships in the **Glasgow norms dataset**.

Specifically we are interested in the following questions:

1. [Is there a relationship between part of speech (noun/verb) and gender?](#t_test)
2. [Are polysemous words more concrete than non-polysemous words?](#mw_test)
3. [Is there a relationship between part of speech (noun/verb) and polysemy?](#Chi)
5. [Can imageability be used to predict word concreteness?](#reg)

In [ ]:
# Load the dataset in python

df = pd.read_csv("words_glasgow.csv", skipinitialspace=True, sep=',')
df.head()

Let's add a variable for part of speech (noun, verb, other).

In [ ]:
# Add some parts of speech tags using nltk
from nltk.tag import pos_tag
pos_tags = [x[1] for x in pos_tag(list(df.word.values))]
df['pos'] = pos_tags

# Classify them as noun, verb, adjective, or other
df['POS'] = 0
df.loc[df['pos'].str.startswith('N'), 'POS'] = 1
df.loc[df['pos'].str.startswith('V'), 'POS'] = 2

df.head()

Now let's send our dataframe to the R environment and have a look at the data before doing some statistical testing.

In [ ]:
%%R -i df

df_num <- df %>% dplyr::select(-word, -pos, -web_corpus_freq) %>% na.omit()

In [ ]:
%%R

# View correlation plot
df_cor <- cor(df_num)
corrplot(df_cor, type = "lower")

In [ ]:
%%R

# View histograms
par(mfrow = c(3,4))
for (i in 1:12) {
  hist(df_num[,i], xlab = colnames(df_num)[i], main = "")
}
par(mfrow=c(1,1))

<a id='t_test'></a>
### 3.A Is there a relationship between part of speech (noun/verb) and gender? ([to top](#top))

#### What test should we use?

**Step 1: Understand type of predictor and outcome variables.**

- Predictor(s): 1 **Categorical** predictor (part of speech) with 2 groups (noun, verb)
- Outcome(s): 1 **Quantitative** outcome (gender)

For this type of data, we should use either an **unpaired t-test** (parametric) or a **Mann-Whitney U test** (non-parametric). Whether we use a parametric test or a non-parametric test depends on the sample size (which we can control to some extent) and the assumptions of the sampled data. If the sample size is not too small, and the assumptions are met, we prefer a parametric test.

**Step 2: Check the assumptions of the sampled data.**
- The outcome data are approximately **normally distributed** for both groups 
- The outcome data have **similar variances** across groups

Now let's get a **sample of our data**, have a look at it, and check our assumptions.

In [ ]:
%%R

set.seed(100)
n <- 100
N_gen_samp <- sample_n(df %>% filter(POS == 1) %>% dplyr::select(word, POS, gender) %>% na.omit(), n)
V_gen_samp <- sample_n(df %>% filter(POS == 2) %>% dplyr::select(word, POS, gender) %>% na.omit(), n)

NV_gen_samp <- rbind(N_gen_samp, V_gen_samp)

ggplot(NV_gen_samp, aes(x=POS %>% as.factor(), y=gender)) + geom_boxplot()

In [ ]:
%%R

# Are gender for nouns and gender for verbs approximately normally distributed? 
# test of normality: p-value <= 0.05 suggests data is NOT normally distributed
print(shapiro.test(N_gen_samp$gender))
print(shapiro.test(V_gen_samp$gender))

# Do gender for nouns and gender for verbs have similar variances?
# test of homogeneity of variances: p-value <= 0.05 suggests variances are equal
print(var.test(gender ~ POS %>% as.factor(), data = NV_gen_samp))

#### We should use the unpaired t-test
Since both assumptions are satisfied and the sample size is not small, we choose the **unpaired t-test** (parametric test). Since we have chosed a parametric test, we are interested in a **difference in means**.

Now we can define our null and alternative hypotheses:
- $H_{0}$: Gender for nouns **does not differ from** gender for verbs
- $H_{A}$: Gender for nouns **differs from** gender for verbs

Since we do not have a specific hypothesis about the direction of the relationship, this is a **two-sided** test.

In [ ]:
%%R

# t-test: p-value <= 0.05 suggests we should reject the null hypothesis
T_Test <- t.test(gender ~ POS,
                data = NV_gen_samp,
                alternative = "two.sided",
                paired = F)
T_Test

#### Conclusion
The p-value is greater than 0.05, which is our significance level, so we **fail to reject the null hypothesis**. Therefore we conclude that **there is no difference between the gender of nouns and verbs**.

<a id='MWU_test'></a>
### 3.B Are polysemous words more concrete than non-polysemous words? ([to top](#top))

#### What test should we use?

**Step 1: Understand type of predictor and outcome variables.**

- Predictor(s): 1 **Categorical** predictor (polysemy) with 2 groups (polysemous, non-polysemous)
- Outcome(s): 1 **Quantitative** outcome (concreteness)

For this type of data, we should use either an **unpaired t-test** (parametric) or a **Mann-Whitney U test** (non-parametric).

**Step 2: Check the assumptions of the sampled data.**
- The outcome data are approximately **normally distributed** for both groups 
- The outcome data have **similar variances** across groups

Now let's get a **sample of our data**, have a look at it, and check our assumptions.

In [ ]:
%%R

set.seed(100)
n <- 100
P1_con_samp <- sample_n(df %>% filter(polysemy == 1) %>% dplyr::select(word, polysemy, concreteness) %>% na.omit(), n)
P0_con_samp <- sample_n(df %>% filter(polysemy == 0) %>% dplyr::select(word, polysemy, concreteness) %>% na.omit(), n)

P_con_samp <- rbind(P1_con_samp, P0_con_samp)

ggplot(P_con_samp, aes(x=polysemy %>% as.factor(), y=concreteness)) + geom_boxplot()

In [ ]:
%%R

# Are gender for nouns and gender for verbs approximately normally distributed? 
# test of normality: p-value <= 0.05 suggests data is NOT normally distributed
print(shapiro.test(P1_con_samp$concreteness))
print(shapiro.test(P0_con_samp$concreteness))

# Do gender for nouns and gender for verbs have similar variances?
# test of homogeneity of variances: p-value <= 0.05 suggests variances are equal
print(var.test(concreteness ~ polysemy %>% as.factor(), data = P_con_samp))

#### We should use the Mann-Whitney U test

Since the assumptions are not satisfied, we choose the **Mann-Whitney U test** (non-parametric test).

Now we can define our null and alternative hypotheses:
- $H_{0}$: Concreteness for polysemous words **does not differ from** concreteness for non-polysemous words
- $H_{A}$: Concreteness for polysemous **is greater than** concreteness for non-polysemous words

Since we have a specific hypothesis about the direction of the relationship, this is a **one-sided** test.

In [ ]:
%%R

MWU_test <- wilcox.test(P1_con_samp$concreteness,
                       P0_con_samp$concreteness,
                       alternative = "greater",
                       paired=FALSE)
MWU_test

#### Conclusion
The p-value is less than 0.05, which is our significance level, so we **reject the null hypothesis**. Therefore we conclude that **polysemous words are more concrete than non-polysemous words**.

<a id='Chi'></a>
### 3.C Is there a relationship between part of speech (noun/verb) and polysemy?

#### What test should we use?

**Step 1: Understand type of predictor and outcome variables.**

- Predictor(s): 1 **Categorical** predictor (part of speech) with 2 groups (noun, verb)
- Outcome(s): 1 **Categorical** outcome (polysemy) with 2 groups (polysemous, non-polysemous)

#### We should use the Chi Square test of Independence

For this type of data, we should use a **Chi Square test of Independence** since both the predictor and outcome variables are categorical. This is a non-parametric test. There is no parametric version of this test.

Now we can define our null and alternative hypotheses:
- $H_{0}$: Part of speech and polysemy are **independent**.
- $H_{A}$: Part of speech and polysemy are **not independent**.

**Step 2: Check the assumptions of the sampled data.**
- Data must be frequencies in a contingency tables
- Expected value of cells should be 5 or greater in at least 80% of cells

If the assumption of the expected values of the cells being 5 or greater is not met, we should take a larger sample if possible.

Now let's get a **sample of our data**, have a look at it, and check our assumptions. 

In [ ]:
%%R

set.seed(100)
n  <- 200
POS_poly_samp <- sample_n(df %>% filter(POS != 0) %>% dplyr::select(word, polysemy, POS) %>% na.omit(), n)
POS_poly_samp <- POS_poly_samp %>% mutate_at(c('POS', 'polysemy'), as.factor)

# make contingency table
tab <- table(POS_poly_samp %>% dplyr::select(-word))

# Chi Square test
Chi_test <- chisq.test(tab)

# Table of expected values
# If at least 20% of cells are less than 5, take a bigger sample
Chi_test$expected

In [ ]:
%%R
Chi_test

#### Conclusion
The p-value is greater than 0.05, which is our significance level, so we **fail to reject the null hypothesis**. Therefore we conclude that **there is no relationship between part of speech (noun/verb) and polysemy**.

<a id='reg'></a>
### 3.D Can imageability be used to predict word concreteness?

#### What test should we use?

**Step 1: Understand type of predictor and outcome variables.**

- Predictor(s): 1 **Quantitative** predictor (imageability)
- Outcome(s): 1 **Quantitative** outcome (concreteness)

#### We should use linear regression

To find a relationship between a quantitative predictor and a quantitative outcome, we could use a correlation analysis, but a correlation analysis does not assume any direction of the relationship. A regression analysis on the other hand **assumes a direction of the relationship**. In addition to quantifying the relationship between quantitative variables, regression models can be **used to make predictions** about outcome variable given new values of the predictor variable.

**Simple linear regression vs. multiple linear regression**

Simple linear regression involves only 1 predictor variable (and can be easily visualized in 2 dimensions), while multiple linear regression involves 2 or more predictor variables. The goal of regression is to find a **line of best fit** that best describes the relationship between the predictor(s) and outcome such that the **error is minimized**, typically, the **sum of squared error (SSE)** is minimized.

In **simple linear regression**, the line of best fit is a function $Y = mX + b$ where $Y$ is the outcome (the variable we want to predict) and $X$ is the predictor. The goal is to find $m$ the slope and $b$ the intercept of the line to minimize the error. The coefficient $m$ represents the relationship between the predictor and the outcome. The **null hypothesis** is the the **coefficient is equal to zero**, reflecting no relationship between the predictor and the outcome.

In **multiple linear regression** works in the same way as simple linear regression but the line of best fit is a function the line of best fit is a function $Y = m_1X_1 + m_2X_2 ... + m_nX_n + b$, so it has multiple predictors and coefficients to be estimated.

**What output is produced by a regression model?** 

A regression model fit to data produces the following output:
- **coefficient estimates** with corresponding p-values reflecting the relationship between predictor(s) and outcome
- **r-squared** value reflecting the proportion of variance explained by the model
- **f-statistic** with corresponding p-value reflecting how well the model fits the data overall

Now we can define our null and alternative hypotheses:
- $H_{0}$: There is no relationships between imageability and concreteness ($m$ = 0)
- $H_{A}$: There is a positive relationship between imageability and concreteness ($m$ > 0)

**Step 2: Check the assumptions of the sampled data.**
- Linear relationship between predictor(s) and outcome
- Residuals approximately normally distributed
- Variance of residuals approximately uniformly distributed
- Low correlation among predictors (low multicollinearity)

The first assumption can be checked with a simple scatterplot. The second and third should be checked after the model is fit, since they are calculated on residuals, which is the difference between the actual outcomes and the predicted outcomes (falling on the line of best fit). The last assumption needs to be checked in the case of multiple linear regression in which there are multiple predictors.

Now let's get a **sample of our data**, and view the scatterplot.

In [ ]:
%%R

set.seed(100)
n  <- 200
reg_samp <- sample_n(df_num, n)

plot(concreteness ~ imageability, data = reg_samp)

There is clearly a **linear relationship** between the two variables.

Now let's fit the linear model.

In [ ]:
%%R

model <- lm(concreteness ~ imageability, data = reg_samp)
summary(model)

In [ ]:
%%R

# View the plot with the line of best fit
plot(concreteness ~ imageability, data = reg_samp)
abline(model, col = "blue")

The models looks like a **good fit**, judging by the low p-value for the coefficient, the high R-squared, and the low p-value for the F-statistic, but we still **need to check the other assumptions**, since the estimates of the model depend on these assumptions.

In [ ]:
%%R

par(mfrow=c(2,2))
plot(model)
par(mfrow=c(1,1))

#### Conclusion

The residuals appear to be approximately normally distributed and the variance of residuals appears to be approximately uniformly distributed.

All of our assumptions are satisfied, and the model parameter estimates are significant, therefore we can conclude that **imageability can be used to predict word concreteness**.

#### What if we do not know what variables are best for predicting a variable?

In many cases, we do not know which combination of predictor variables is best for predicting an outcome.

If we have too few predictors, we are not utilizing the full potential of our dataset to predict an outcome, but usually when it comes to regression models, less is more, because **too many predictor variables can lead to overfitting**. The best model maximizes **parsimony**, striking a balance between model simplicity and the ability of the model to explain the data accurately.

The parsimony of a model can be measured with measures like:
- **AIC** which rewards models for how well they explain the data while penalizing them for their complexity
- **BIC** which imposes a stronger penalty for model complexity than AIC

Considering two different models with different numbers of predictors, we can use the AIC and BIC to see which model is more parsimonious (lower values are better).

In [ ]:
%%R
model1 <- lm(concreteness ~ imageability + length, data = reg_samp)
model2 <- lm(concreteness ~ imageability + length + aoa, data = reg_samp)

In [ ]:
%%R
print(AIC(model1))
print(AIC(model2))

In [ ]:
%%R
print(BIC(model1))
print(BIC(model2))

According to the AIC the more complex model is better, while according to the BIC the simpler model is better. So the choice of AIC vs. BIC for assessing a model should be made considering what the model will be used for.

There are also **methods for model selection** that find the best model (using AIC or BIC) from different combinations of predictors. They include:
- **Forwards selection** which starts with no predictors in the model and adds predictors until the best model is reached
- **Backwards selection** which starts with all predictors in the model and removes predictors until the best model is reached
- **Stepwise selection** which is a combination of forward and backward selection

Here we see an example of stepwise selection to select a model for predicting concreteness.

In [ ]:
%%R

# Fit the full model 
full.model <- lm(concreteness ~., data = reg_samp %>% dplyr::select(-polysemy, -POS))

# Stepwise regression model
step.model <- stepAIC(full.model, direction = "both", 
                      trace = FALSE)
summary(step.model)